In [29]:
# accepted constraints pattern:
#   t1.<col_name> <bool operator> t2.<col_name>
#   t1.<col_name> <bool operator> <scalar_value>

!pip install pandas

You should consider upgrading via the '/home/alessandro/.asdf/installs/python/3.10.5/bin/python3.10 -m pip install --upgrade pip' command.


In [30]:
import pandas
from functools import reduce
import re

In [31]:
def check_constraint_sintax(constraint):
    pattern1 = r"t1\..*\s+[<>=!]+\s+t2\..*"
    pattern2 = r"t1\..*\s+[<>=!]+\s+(?:\d+|\'.*\'|\".*\")"          # TODO: retirar t1.* right side 

    if re.match(pattern1, constraint) or re.match(pattern2, constraint):
        return True
    else:
        print(f"invalid constraint sintax: {constraint}")
        return False

In [32]:
def has_scalar_value_compare(constraint):
    pattern = r"t1\..*\s+[<>=!]+\s+t2\..*"
    return False if re.match(pattern, constraint) else True

In [33]:
def to_number_else_str(input_string):
    if input_string.isdigit() or (input_string[0] == '-' and input_string[1:].isdigit()):
        return int(input_string)
    else:
        return input_string

def get_col_name(alias, constraint):
    pattern = re.escape(alias) + r'\.(\w+)'
    correspondences = re.findall(pattern, constraint)
    if correspondences:
        return correspondences[0]
    return None

In [34]:
operators_fn = {
  '>': lambda scalar, df, col: scalar > df[col],
  '<': lambda scalar, df, col: scalar < df[col],
  '=': lambda scalar, df, col: scalar == df[col],
  '>=': lambda scalar, df, col: scalar >= df[col],
  '<=': lambda scalar, df, col: scalar <= df[col],
  '<>': lambda scalar, df, col: scalar != df[col],
}

def gen_pandas_condition(df, lcs, col_alias):
    
    scalar = to_number_else_str(lcs.split(' ')[0])
    operator = lcs.split(' ')[1]
    col = get_col_name(col_alias, lcs)

    return operators_fn[operator](scalar, df, col)

In [35]:
def invert_expression(expression):
    operators = {
        '<': '>',
        '>': '<',
        '<=': '>=',
        '>=': '<=',
        '!=': '!=',
        '=': '='
    }

    pattern = r'(t1\.\w+)\s*([><=]+)\s*(\d+)'

    substitution = lambda match: f'{match.group(3)} {operators.get(match.group(2), match.group(2))} {match.group(1)}'

    new_expression = re.sub(pattern, substitution, expression)
    return new_expression

In [36]:
def switch_t1_by_scalar(constraint, value):
    pattern = r't1\.\w+'
    new_expression = re.sub(pattern, str(value), constraint)
    return new_expression

def resolve_left_side_constraints(df_tuple, constraints):
  lcs = []
  for cs in constraints:
    t1_col_name = get_col_name('t1', cs)
    t1_value = df_tuple[t1_col_name]
    new_cs = switch_t1_by_scalar(cs, t1_value)
    lcs.append(new_cs)

  return lcs

In [37]:
########################## EXEC ##########################

In [38]:
employes_df = pandas.read_csv('employes.csv')
departments_df = pandas.read_csv('departments.csv')

df = pandas.merge(employes_df, departments_df, how='left', on='departamento_id')

In [39]:
print(employes_df, end='\n\n')
print(departments_df, end='\n\n')
print(df)

   id   ano  departamento_id  salario
0   1  2023                3    50000
1   2  2023                2   100000
2   3  2022                4    75000
3   4  2024                1    55000
4   5  2023                1    70000
5   6  2023                5    48000

   departamento_id departamento   bonus
0                1    Marketing    3500
1                2       Vendas  350000
2                3   Tecnologia  100000
3                4    Operações   80000
4                5   Financeiro    9000

   id   ano  departamento_id  salario departamento   bonus
0   1  2023                3    50000   Tecnologia  100000
1   2  2023                2   100000       Vendas  350000
2   3  2022                4    75000    Operações   80000
3   4  2024                1    55000    Marketing    3500
4   5  2023                1    70000    Marketing    3500
5   6  2023                5    48000   Financeiro    9000


In [40]:
with open('constraints.txt', 'r') as file:
    first_line = file.readline().strip()

constraints = first_line.split(',')

In [41]:
print(constraints)
print()
print(df)

['t1.salario > t2.salario', 't1.ano < t2.ano']

   id   ano  departamento_id  salario departamento   bonus
0   1  2023                3    50000   Tecnologia  100000
1   2  2023                2   100000       Vendas  350000
2   3  2022                4    75000    Operações   80000
3   4  2024                1    55000    Marketing    3500
4   5  2023                1    70000    Marketing    3500
5   6  2023                5    48000   Financeiro    9000


In [42]:
for cs in constraints:
  check_constraint_sintax(cs)

In [43]:
scalar_constraints = list(filter(has_scalar_value_compare, constraints))
relational_constraints = [cs for cs in constraints if cs not in scalar_constraints]

In [44]:
print(scalar_constraints)
print(relational_constraints)

[]
['t1.salario > t2.salario', 't1.ano < t2.ano']


In [45]:
df_filtered_by_scalar_ex = df

if len(scalar_constraints):
  constraints_scalar_left_side = [ invert_expression(sc) for sc in  scalar_constraints]
  pandas_scalar_conditions = [ gen_pandas_condition(df, csl, 't1') for csl in constraints_scalar_left_side ]
  df_filtered_by_scalar_ex = df[reduce(lambda x, y: x & y, pandas_scalar_conditions)]

  print(scalar_constraints)
  print(constraints_scalar_left_side)


In [46]:
print(df_filtered_by_scalar_ex)

   id   ano  departamento_id  salario departamento   bonus
0   1  2023                3    50000   Tecnologia  100000
1   2  2023                2   100000       Vendas  350000
2   3  2022                4    75000    Operações   80000
3   4  2024                1    55000    Marketing    3500
4   5  2023                1    70000    Marketing    3500
5   6  2023                5    48000   Financeiro    9000


In [47]:
tuples_qtd = df.shape[0]

df['violations'] = 0
df['targets'] = [[] for _ in range(tuples_qtd)]

In [48]:
if bool(len(relational_constraints)):
  for idx, line in df_filtered_by_scalar_ex.iterrows():
    left_resolved_constraints = resolve_left_side_constraints(line, relational_constraints)
    pandas_conditions = [ gen_pandas_condition(df, lcs, 't2') for lcs in left_resolved_constraints ]
    pandas_conditions_set = reduce(lambda x, y: x & y, pandas_conditions)
    violations_ids = df[pandas_conditions_set]['id'].to_list()
    df.at[idx, 'violations'] = len(violations_ids)
    df.at[idx, 'targets'] = violations_ids

else:
  for idx, line in df_filtered_by_scalar_ex.iterrows():
    df.at[idx, 'violations'] = 1
    df.at[idx, 'targets'] = [line['id']]

print(constraints, end='\n\n')
print(df)

['t1.salario > t2.salario', 't1.ano < t2.ano']

   id   ano  departamento_id  salario departamento   bonus  violations  \
0   1  2023                3    50000   Tecnologia  100000           0   
1   2  2023                2   100000       Vendas  350000           1   
2   3  2022                4    75000    Operações   80000           4   
3   4  2024                1    55000    Marketing    3500           0   
4   5  2023                1    70000    Marketing    3500           1   
5   6  2023                5    48000   Financeiro    9000           0   

        targets  
0            []  
1           [4]  
2  [1, 4, 5, 6]  
3            []  
4           [4]  
5            []  
